In [1]:
import pandas as pd
import json
import numpy as np

In [80]:
f = open("../results/test.json")
data = json.load(f)
mean_time_survival = 701
#data.keys()
# lists to arrays
labels = np.array(data['labels'])
predictions = np.array(data['predictions'])
stds = np.array(data['stds'])

# Compute values in real scale
labels_real = - mean_time_survival*np.log(
        np.clip(1-labels, a_min=1e-5, a_max=1))
predictions_real = - mean_time_survival*np.log(
        np.clip(1-predictions, a_min=1e-5, a_max=1))
stds_real =mean_time_survival * stds/(1 - predictions)
perfect_std_real = mean_time_survival * stds/(1 - labels)
# compute MAE and MAE between elements in real scale
mae = np.abs(labels-predictions)
mae_real = np.abs(labels_real-predictions_real)

# compute correlation coefficients:
corr_mae_stds = np.corrcoef(mae, stds)
corr_mae_stds_real = np.corrcoef(mae_real, stds_real)

# compute correlation between labels and predicttions:
corr_label_prediction = np.corrcoef(labels, predictions)
corr_label_prediction_real = np.corrcoef(labels_real, predictions_real)

# compute r2 score:
r2 = r2_score(labels, predictions)
r2_real =r2_score(labels_real, predictions_real)


In [60]:
- mean_time_survival*np.log(
        np.clip(1-0.5, a_min=1e-5, a_max=1))

485.89617357252166

In [75]:
corr_mae_stds

array([[1.        , 0.27889015],
       [0.27889015, 1.        ]])

In [76]:
corr_mae_stds_real

array([[1.       , 0.3765505],
       [0.3765505, 1.       ]])

In [78]:
corr_label_prediction_real

array([[1.        , 0.52902662],
       [0.52902662, 1.        ]])

In [81]:
r2

0.4246883745047396

In [82]:
r2_real

0.18057203564396984

In [83]:
from sklearn.metrics import f1_score, accuracy_score, balanced_accuracy_score, roc_auc_score, r2_score
def compute_metrics(labels, predictions_real,predictions, thresh = 701):
    metrics = dict()
    new_thresh = 1 - np.exp(-thresh/701)
    
    new_labels = (labels>=thresh).astype(int)
    new_predictions = (predictions_real>=thresh).astype(int)
    metrics["f1_score"] = f1_score(new_labels, new_predictions, average = None).tolist()
    metrics["accuracy"] = accuracy_score(new_labels, new_predictions)
    metrics["balanced_accuracy"] = balanced_accuracy_score(new_labels, new_predictions)
    metrics["auc"] = roc_auc_score(new_labels, predictions-new_thresh+0.5).tolist()
    return metrics

In [88]:
compute_metrics(labels_real, predictions_real,predictions,  365)

{'f1_score': [0.7322976750769666, 0.7410222176577318],
 'accuracy': 0.7367322081085784,
 'balanced_accuracy': 0.7370445255995883,
 'auc': 0.8173160981766858}